In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy
import math

In [24]:
# Defining Parameters
TARGET_SIZE = (256, 256)
COLOR_MODE = 'grayscale'
BATCH_SIZE = 5

In [25]:
# Load Images
data_gen = ImageDataGenerator(rescale=1.0/255)
train_it = data_gen.flow_from_directory("Covid19-dataset/train", target_size=TARGET_SIZE, color_mode=COLOR_MODE, class_mode='sparse', batch_size=BATCH_SIZE)
validate_it = data_gen.flow_from_directory("Covid19-dataset/test", target_size=TARGET_SIZE, color_mode=COLOR_MODE, class_mode='sparse', batch_size=BATCH_SIZE)


print(train_it.image_shape)
print(train_it.num_classes)
print(train_it.samples)
print(validate_it.samples)

Found 253 images belonging to 3 classes.
Found 66 images belonging to 3 classes.
(256, 256, 1)
3
253
66


In [26]:
# Create Model
model = Sequential()
model.add(layers.Input(shape=(256, 256, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │       196,611 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 196,611 (768.01 KB)

 Trainable params: 196,611 (768.01 KB)

 Non-trainable params: 0 (0.00 B)

None


In [27]:
# Fit and Evaluate Model
model.fit(
    train_it, steps_per_epoch=math.ceil(train_it.samples/BATCH_SIZE), 
    epochs=10, validation_data=validate_it, validation_steps=math.ceil(validate_it.samples/BATCH_SIZE)
)

Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5088 - loss: 17.7415 - val_accuracy: 0.7727 - val_loss: 4.3818
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7998 - loss: 3.2007 - val_accuracy: 0.6818 - val_loss: 4.2094
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 117us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8596 - loss: 2.7985 - val_accuracy: 0.8485 - val_loss: 2.1574
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
25/51 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9264 - loss: 1.0725 

2024-10-29 19:00:19.171335: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9175 - loss: 1.0046 - val_accuracy: 0.5758 - val_loss: 11.6357
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8444 - loss: 2.4625 - val_accuracy: 0.7727 - val_loss: 3.9109
Epoch 10/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 117us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
